# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [31]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [32]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [33]:
from pyteal import *

In [34]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [35]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [36]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19804483


## Modesty: A more complicated contract
* A donation to **Bob** that limits the amount that can be withdrawn in one transaction

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [37]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [39]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=3)
print(modest_teal)

#pragma version 3
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 1000000
<=
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [40]:
Modest = algod_client.compile(modest_teal)
Modest

{'hash': 'B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ',
 'result': 'AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhA='}

##### Step 4: Alice funds and deploys the smart signature

In [42]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804537.
Waiting for round 19804537 to finish.
Waiting for round 19804538 to finish.
Transaction 2CIQ7I66YGNGTXNR7E6TCXZJZGM2XJFMSIUXTUHEBG6ZYLYGOO4Q confirmed in round 19804539.


##### Step 5: Alice informs Bob

In [44]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhA=
Address of smart signature:  B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ


In [45]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])

https://testnet.algoexplorer.io/address/B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ


#### Step 6: Bob proposes a transaction that would withdraw too much

In [46]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 1 192 132 61 38 1 32 118 164 184 189 92 71 37 106 56 88 49 148 115 208 69 74 75 215 68 229 95 253 135 115 101 196 195 243 209 223 1 198 49 7 40 18 49 8 34 14 16] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:pay Header:{_struct:{} Sender:B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ Fee:{Raw:1000} FirstValid:19804556 LastValid:19805556 Note:[] GenesisID:testnet-v1.0 GenesisHash:JBR3KGFEWPEE5SAQ6IWU6EEBZMHXD4CZU6WCBXWGF57XBZIJHIRA Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAA

#### Step 6: Bob withdraws smaller amount

In [47]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- within limit!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804562.
Waiting for round 19804562 to finish.
Waiting for round 19804563 to finish.
Transaction H4JM3WB273TXR6XHF5OI7A6NXH7IRG3WPKB4ZGJR6MPMP2DCN7TQ confirmed in round 19804564.


# Appendix: why the `Int(int(1e6))`
Pyteal requires the type `pyteal.Int` (with large I). This can only be obtained from Python's `int` type.

In [48]:
a = 1E6
type(a)

float

In [49]:
b = int(1E6)
type(b)

int

In [50]:
c = Int(int(1E6))
type(c)

pyteal.Int

In [51]:
# This is not possible
d = Int(1E6)

TealInputError: invalid input type <class 'float'> to Int